<a href="https://colab.research.google.com/github/SridharSola/Contrastive-learning-FER/blob/main/training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
def adjust_learning_rate(optimizer): 
  for param_group in optimizer.param_groups: 
      param_group["lr"] /= 10.

def accuracy(out, labels):
    _,pred = torch.max(out, dim=1)
    return torch.sum(pred==labels).item()

def training(MTN,emoClassifierU, emoClassifierM, maskClassifier, train_set, test_set, learning_rate, epochs, lamda, alpha, beta, log, model_save_U, model_save_M, f1_mask = 0, f2_mask = 1):
  if torch.cuda.is_available():
    MTN.cuda()
    emoClassifierU.cuda()
    emoClassifierM.cuda()

  MTN.train()
  emoClassifierU.train()
  emoClassifierM.train()
  optimizer = torch.optim.Adam([{'params':MTN.parameters(), 'lr': learning_rate}, #
                                {'params':emoClassifierU.parameters(), 'lr': learning_rate},
                                {'params':emoClassifierM.parameters(), 'lr': learning_rate}
                                ]) #added other paras as list
  logfile = open(log, 'w')
  n_epochs = epochs
  print_every = 5
  u_test_loss_min = np.Inf
  m_test_loss_min = np.Inf
  test_loss_m = []
  test_loss_u = []
  test_acc_maskU = []
  test_acc_maskM = []
  test_acc_emoU = []
  test_acc_emoM = []
  train_loss = []
  train_acc_mask = []
  train_acc_emoU = []
  train_acc_emoM = []
  Ubest_test_acc_mask = 0.0
  Ubest_test_acc_emo = 0.0
  Mbest_test_acc_mask = 0.0
  Mbest_test_acc_emo = 0.0
  total_step = len(train_set)
  lrs = []
  lrs.append(learning_rate)
  mse =nn.MSELoss().cuda()
  
  bce = torch.nn.BCELoss()
  ceU = nn.CrossEntropyLoss()
  ceM = nn.CrossEntropyLoss()
  m = nn.Sigmoid()
  
  emoLoss = nn.CrossEntropyLoss().cuda()
  maskLoss = nn.BCELoss().cuda()
  
  
  print('\nTraining starting:\n', file = logfile)
  print('\nTraining starting:\n')
  for epoch in range(1, n_epochs+1):
      running_loss = 0.0
      correct_u_mask = 0
      correct_u_emo = 0
      correct_m_mask = 0
      correct_m_emo = 0
      total=0
      print(f'Epoch {epoch}\n', file = logfile)
      print(f'Epoch {epoch}\n')
      if epoch == 20 or epoch == 28 or epoch==36 or epoch == 50 or epoch == 60:     
          #scheduler.step()
          adjust_learning_rate(optimizer)
          lrs.append(optimizer.param_groups[0]["lr"])
          print(f'Updated lr: {lrs[-1]}\n', file = logfile)
          print(f'Updated lr: {lrs[-1]}\n')
      for batch_idx, (udata_, mdata_, target_) in enumerate(train_set):
          mdata_, udata_, target_ = mdata_.to(device), udata_.to(device), target_.to(device)
          #print(mdata_.shape)
          optimizer.zero_grad()
          target_m = torch.ones(target_.shape[0],1).cuda()
          target_m = target_m * f2_mask
          #print(target_m.shape)
          target_u = torch.ones(target_.shape[0], 1).cuda()
          target_u = target_u * f1_mask
          #First call to backbone net for unmasked images--------(1)
          #p = net(udata_)
          u_mv, u_ev = MTN(udata_)
          #print(u_mv.shape, u_ev.shape)
          #print(u_emo_probs)
          
          #Second call to backbone net for maskd images---------(2)
          m_mv, m_ev = MTN(mdata_)
          #print(m_mv.shape)
          u_emo_probs = emoClassifierU(u_ev)
          m_emo_probs = emoClassifierM(m_ev)
          #Losses
          Lmse = mse(u_ev, m_ev)
          emo_lossU = ceU(u_emo_probs, target_)
          emo_lossM = ceM(m_emo_probs, target_)
          #Total loss is weighted combiation of above losses
          loss = alpha*Lmse + beta*(emo_lossM + emo_lossU)

          loss.backward()
          optimizer.step()
          
          #Duplicated following as needed
          running_loss += loss.item()

          _,u_emo_pred = torch.max(u_emo_probs, dim=1)
          #print(u_emo_pred)
          correct_u_emo += torch.sum(u_emo_pred==target_).item()
          _,m_emo_pred = torch.max(m_emo_probs, dim=1)
          correct_m_emo += torch.sum(m_emo_pred==target_).item()
          

          total += target_.size(0)
          if (batch_idx) % 20 == 0:
              print ('Epoch [{}/{}], Step [{}/{}], Loss(Unmasked): {:.4f},  Loss(Masked): {:.4f}, MSE : {:.4f}, Combined loss: {:.4f} '       
                    .format(epoch, n_epochs, batch_idx, total_step,  emo_lossU.item(), emo_lossM.item(), Lmse.item(),loss.item()), file = logfile)
              print ('Epoch [{}/{}], Step [{}/{}], Loss(Unmasked): {:.4f},  Loss(Masked): {:.4f}, MSE : {:.4f}, Combined loss: {:.4f} ' 
                    .format(epoch, n_epochs, batch_idx, total_step,  emo_lossU.item(), emo_lossM.item(), Lmse.item(),loss.item()))
      train_acc_mask.append(100 * (correct_u_mask + correct_m_mask) / (total*2))
      train_acc_emoU.append(100 * correct_u_emo / total)
      train_acc_emoM.append(100 * correct_m_emo / total)
      train_loss.append(running_loss/total_step)
      print(f'\ntrain-loss: {np.mean(train_loss):.4f}', file = logfile)
      print(f'\nTrain accuracies: \n FER on Unmasked: {(100 * correct_u_emo / total):.4f}, FER on Masked: {(100 * correct_m_emo / total):.4f}', file = logfile)
      print(f'\ntrain-loss: {np.mean(train_loss):.4f}')
      print(f'\nTrain accuracies: \n FER on Unmasked: {(100 * correct_u_emo / total):.4f}, FER on Masked: {(100 * correct_m_emo / total):.4f}') 
      u_batch_loss = 0
      m_batch_loss = 0
      total_t=0
      u_mask_correct_t=0
      m_mask_correct_t=0
      u_emo_correct_t = 0
      m_emo_correct_t = 0
      #Testing
      with torch.no_grad():
          MTN.eval()
          emoClassifierU.eval()
          emoClassifierM.eval()

          for udata_t, mdata_t, target_t in (test_set):
              udata_t, mdata_t, target_t = udata_t.to(device), mdata_t.to(device), target_t.to(device)
              target_m = torch.ones(target_t.size(0),1).cuda()
              target_m = target_m * f2_mask
              target_u = torch.ones(target_t.size(0), 1).cuda()
              target_u = target_u * f1_mask
              u_mvt, u_evt = MTN(udata_t)
              u_emo_probs_t = emoClassifierU(u_evt)

              m_mvt, m_evt = MTN(mdata_t)
              m_emo_probs_t = emoClassifierM(m_evt)
              _,u_emo_pred_t = torch.max(u_emo_probs_t, dim=1)
              u_emo_correct_t += torch.sum(u_emo_pred_t==target_t).item()
              _,m_emo_pred_t = torch.max(m_emo_probs_t, dim=1)
              m_emo_correct_t += torch.sum(m_emo_pred_t==target_t).item()
              total_t += target_t.size(0)

          test_acc_emoU.append(100 * u_emo_correct_t/total_t)
          test_acc_emoM.append(100 * m_emo_correct_t/total_t)
          test_loss_u.append(u_batch_loss/len(test_set))
          test_loss_m.append(m_batch_loss/len(test_set))
          network_learnedU = u_batch_loss < u_test_loss_min
          network_learnedM = m_batch_loss < m_test_loss_min
          print(f'\nUnmasked test loss: {np.mean(test_loss_u):.4f}', file = logfile)
          print(f'Unmasked Test Accuracies: \n FER: {(100 * u_emo_correct_t/total_t):.4f}', file = logfile)     #Mask Detection: {(100 * u_mask_correct_t/total_t):.4f},
          print(f'Unmasked test loss: {np.mean(test_loss_u):.4f}')
          print(f'Unmasked Test Accuracies: \n FER: {(100 * u_emo_correct_t/total_t):.4f}')
          
          print(f'\nMasked test loss: {np.mean(test_loss_m):.4f}', file = logfile)
          print(f'Masked Test Accuracies: \n FER: {(100 * m_emo_correct_t/total_t):.4f}', file = logfile)   #Mask Detection: {(100 * m_mask_correct_t/total_t):.4f},
          print(f'Masked test loss: {np.mean(test_loss_m):.4f}')
          print(f'Masekd Test Accuracies: \n FER: {(100 * m_emo_correct_t/total_t):.4f}')
          """
          if Ubest_test_acc_mask < test_acc_maskU[-1]:
            #Updating the best test_accuracy obtained for MD on U set
            print("Best Mask Detection Test Accuracy Updated for Unmasked set", file = logfile)
            print("Best Mask Detection Test Accuracy Updated for Unmasked set")
            Ubest_test_acc_mask = test_acc_maskU[-1]
            """
          if Ubest_test_acc_emo < test_acc_emoU[-1]:
            #Updating the best test_accuracy obtained
            print("Best FER Test Accuracy Updated for unmasked set", file = logfile)
            print("Best FER Test Accuracy Updated for unmasked set")
            Ubest_test_acc_emo = test_acc_emoU[-1]
          """
          if Mbest_test_acc_mask < test_acc_maskM[-1]:
            #Updating the best test_accuracy obtained for MD on M set
            print("Best Mask Detection Test Accuracy Updated for masked set", file = logfile)
            print("Best Mask Detection Test Accuracy Updated for masked set")
            Mbest_test_acc_mask = test_acc_maskM[-1]
          """
          if Mbest_test_acc_emo < test_acc_emoM[-1]:
            #Updating the best test_accuracy obtained
            print("Best FER Test Accuracy Updated for masked set", file = logfile)
            print("Best FER Test Accuracy Updated for masked set")
            Mbest_test_acc_emo = test_acc_emoM[-1]
          
          if network_learnedU:
              u_test_loss_min = u_batch_loss
              torch.save(net.state_dict(), model_save_U)
              print('Improvement-Detected for unmasked, save-model to drive', file = logfile)
              print('Improvement-Detected for unmasked, save-model to drive')
          if network_learnedM:
              m_test_loss_min = m_batch_loss
              torch.save(net.state_dict(), model_save_M)
              print('Improvement-Detected for masked, save-model to drive', file = logfile)
              print('Improvement-Detected for masked, save-model to drive')


      MTN.train()
      emoClassifierU.train()
      emoClassifierM.train()

  print("The best Mask Detection test accuracy obtained on Unmasked images was: ", Ubest_test_acc_mask)
  print("The best FER test accuracy obtained on Unmasked images was: ", Ubest_test_acc_emo)
  print("The best Mask Detection test accuracy obtained on Masked images was: ", Mbest_test_acc_mask)
  print("The best FER test accuracy obtained on Masked images was: ", Mbest_test_acc_emo)
  #End of training